In [134]:
from cryptoadvance.specter.rpc import BitcoinCLI, RpcError
import numpy as np
#cli = BitcoinCLI("bitcoin", "secret", "localhost", "18443")
cli = BitcoinCLI("bitcoin", "dqptmms53t11i8vx93sp", "thundroid.local", "8332")
cli.getblockchaininfo()
# To investigate the Bitcoin API, here are some great resources:
# https://bitcoin.org/en/developer-reference#bitcoin-core-apis
# https://chainquery.com/bitcoin-cli
# https://github.com/ChristopherA/Learning-Bitcoin-from-the-Command-Line

{'chain': 'main',
 'blocks': 623012,
 'headers': 623012,
 'bestblockhash': '000000000000000000073ab61877ffe4f4044d6344854268999b1c01954ec7fc',
 'difficulty': 13912524048945.91,
 'mediantime': 1585221456,
 'verificationprogress': 0.9999962824704106,
 'initialblockdownload': False,
 'chainwork': '00000000000000000000000000000000000000000dec58e285b8018625b853dc',
 'size_on_disk': 305527731289,
 'pruned': False,
 'softforks': [{'id': 'bip34', 'version': 2, 'reject': {'status': True}},
  {'id': 'bip66', 'version': 3, 'reject': {'status': True}},
  {'id': 'bip65', 'version': 4, 'reject': {'status': True}}],
 'bip9_softforks': {'csv': {'status': 'active',
   'startTime': 1462060800,
   'timeout': 1493596800,
   'since': 419328},
  'segwit': {'status': 'active',
   'startTime': 1479168000,
   'timeout': 1510704000,
   'since': 481824}},
 'warnings': ''}

Let's look into the result of the getblock-call. It needs a hash as param and we're using the getblockhash-funtion to obtain the hash via the desired blockheight.

In [135]:
print(cli.getblock(cli.getblockhash(2)).keys())
print(list(cli.getblock(cli.getblockhash(1)).values()))

dict_keys(['hash', 'confirmations', 'strippedsize', 'size', 'weight', 'height', 'version', 'versionHex', 'merkleroot', 'tx', 'time', 'mediantime', 'nonce', 'bits', 'difficulty', 'chainwork', 'nTx', 'previousblockhash', 'nextblockhash'])
['00000000839a8e6886ab5951d76f411475428afc90947ee320161bbf18eb6048', 623012, 215, 215, 860, 1, 1, '00000001', '0e3e2357e806b6cdb1f70b54c3a3a17b6714ee1f0e68bebb44a74b1efd512098', ['0e3e2357e806b6cdb1f70b54c3a3a17b6714ee1f0e68bebb44a74b1efd512098'], 1231469665, 1231469665, 2573394689, '1d00ffff', 1, '0000000000000000000000000000000000000000000000000000000200020002', 1, '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f', '000000006a625f06636b8bb6ac7b960a8d03705d1ace08b1a19da3fdcc99ddbd']


We will need a way to calculate the subsidy. Here is a function which does that:

In [201]:
def blocksubsidy(height):
    ' see https://gist.github.com/nelruk/d00d785f84e47288b0ab734d4aab5f49 '
    period = int(height / 210000)
    subsidy=50
    for i in range(0,period):
        subsidy = subsidy /2
    return subsidy


We will need to iterate through all the blocks. As the hash of the next block is part of the json, we don't have to use the geblockhash-call. This will make it faster. So here is a simplified version of how it looks like to navigate through the first 1000 blocks. We will also sum up the stock and measure the time this will take:

In [213]:
import time
import csv
from datetime import datetime
#blocks = cli.getblockchaininfo()["blocks"]
# Lets' only iterate 1000 blocks
blocks = 1000
tic = time.perf_counter()
hash = cli.getblockhash(1)
stock = 0
for i in range(0, 1000):
    block = cli.getblock(hash)
    stock = stock + blocksubsidy(i)
    hash = block['nextblockhash']

toc = time.perf_counter()
print(f"created the array in {toc - tic:0.4f} seconds")
print(f"In {blocks} blocks, btc {stock} has been created")

created the array in 3.6843 seconds
In 1000 blocks, btc 50000 has been created


So now we'll write out the stock for each day. For that, we'll check for each block, whether i has been created on a different day than the former block.

In [211]:
import time
import csv
from datetime import datetime
blocks = cli.getblockchaininfo()["blocks"]
# ommitting block 0 as it's non standard (18 elements instead of 19 as all the others)
tic = time.perf_counter()
hash = cli.getblockhash(1)
stock = 0
former_block_day = datetime.utcfromtimestamp(cli.getblock(hash)["time"]).strftime('%Y-%m-%d')
with open('days.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(0, 1000):
        block = cli.getblock(hash)
        timestamp = block['time']
        block_day = datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')
        if (block_day != former_block_day):
            writer.writerow([block_day,i,stock])
        stock = stock + blocksubsidy(i)
        former_block_day = block_day
        hash = block['nextblockhash']
    
toc = time.perf_counter()
print(f"created the array in {toc - tic:0.4f} seconds")

created the array in 3.6463 seconds


(17761, 18)